In [ ]:
import os, sys, re, cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
import matplotlib.backends.backend_pdf
from matplotlib.patches import Rectangle

import numpy as np
from numpy import flip, arange, reshape

import seaborn as sns
import pandas as pd
import glob

import scipy
from scipy.signal import correlate2d, fftconvolve
from scipy import ndimage
from scipy.ndimage import maximum_filter
from scipy.spatial import distance_matrix, Delaunay, Voronoi, voronoi_plot_2d

import math
from PIL import Image

from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.cluster import DBSCAN, HDBSCAN

import pointpats

from render2D import grid2D

from utils import *

In [ ]:
prefix = "../data/rendered/soma/"
soma_dates=["0221","0227","0606","1226"]
soma_list=[[7],[8,9,11,20,21],[2],[2,3,5,8,9,10,11,15,18]]
thre=[50,220]
for i in range(len(soma_dates)):
    for j in range(len(soma_list[i])):
        soma=soma_dates[i]+"_"+str(soma_list[i][j])
        img = prefix+soma+".png"
        loc_file = prefix+".txt"
        regions=[650,1500,1000,1850]
        rendered = cv2.imread(img)
        test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
        test1=test[650:1500,1000:1850]
        # plt.figure()
        # plt.imshow(test1)
        #vxs = lmax_loc(loc_file,regions,test1,adapt=55,local=7)[1]
        vxs = lmax_loc(None,None,test1,adapt=55, local=7)[1]
        #histKNN(vxs*16, k=6, bins=range(1,500,10),plot=1)
        #plt.savefig("../result/hist_beforelinks_6_"+soma+".png",bbox_inches='tight',pad_inches=0.0)
        links = generateLinks(vxs*16, thre=thre, plot=0)
        histLinks(vxs*16, links, bins=range(1,500,10))
        plt.savefig("../result/hist_afterlinks_"+soma+".png",bbox_inches='tight',pad_inches=0.0)

In [ ]:
prefix = "../data/rendered/soma/"
soma_dates=["0221","0227","0606","1226"]
soma_list=[[7],[8,9,11,20,21],[2],[2,3,5,8,9,10,11,15,18]]
for i in range(1):
    for j in range(1):
        soma=soma_dates[i]+"_"+str(soma_list[i][j])
        img = prefix+soma+".png"
        loc_file = prefix+".txt"
        regions=[850,1200,1100,1450]
        rendered = cv2.imread(img)
        test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
        test1=test[850:1200,1100:1450]
        # plt.figure()
        plt.imshow(test1)
        #vxs = lmax_loc(loc_file,regions,test1,adapt=55,local=7)[1]
        vxs = lmax_loc(None,None,test1,adapt=55, local=7)[1]
prefix = "../data/rendered/Tcell/"
Tcell_conds=["A","NA"]
Tcell_list=[[1,2,3,5,6,7,10],[1,2,5]]
for i in range(len(Tcell_conds)):
    for j in range(len(Tcell_list[i])):
        tcell=Tcell_conds[i]+str(Tcell_list[i][j])
        img = prefix+tcell+".png"
        loc_file = prefix+".txt"
        regions=[650,1500,650,1500]
        rendered = cv2.imread(img)
        test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
        test1=test[650:1500,650:1500]
        # plt.figure()
        # plt.imshow(test1)
        #vxs = lmax_loc(loc_file,regions,test1,adapt=55,local=7)[1]
        vxs = lmax_loc(None,None,test1,adapt=55, local=7, plot=1)[1]
        histKNN(vxs*16, k=6, bins=range(1,500,10),plot=1)
        plt.savefig("../result/hist_beforelinks_6_"+tcell+".png",bbox_inches='tight',pad_inches=0.0)

prefix = "../data/rendered/Tcell/"
Tcell_conds=["A","NA"]
Tcell_list=[[1,2,3,5,6,7,10],[1,2,5]]
for i in range(1):
    for j in range(1):
        tcell=Tcell_conds[i]+str(Tcell_list[i][j])
        img = prefix+tcell+".png"
        print(img)
        loc_file = prefix+".txt"
        regions=[650,1500,650,1500]
        rendered = cv2.imread(img)
        test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
        test1=test[650:1500,650:1500]
        plt.imshow(test1)
vxs = sim2D()
vxs=vxs*16
n_vx = len(vxs)
links = generateLinks(vxs,thre=220)
tri_list = getTrianglesFromList(convertMatrix2List(links))
t_vxs = getTrianglesCentroid(tri_list, vxs)

v_c = Voronoi_cv(vxs)
v_e = Voronoi_edges(vxs)
connects=np.mean(np.sum(links,0))
tris = len(t_vxs)


thre = [50,220]
#vxs_2D = sim2D(size=16*64,space=93)
#stats_2D = statsForVXS(vxs_2D, plot=0, thre=thre)

g = cv2.getGaussianKernel(3, sigma=1)
kernel = g*g.T
grid_size=64
test_1d=glob.glob("../data/rendered/soma/1D/*.txt")
test_2d=glob.glob("../data/rendered/soma/2D/*.txt")
test_rand=glob.glob("../data/rendered/soma/Rand/*.txt")
stats_1dr=[]
stats_2dr=[]
stats_randr=[]
stats_1ds=[]
stats_2ds=[]
stats_rands=[]
for i in range(len(test_1d)):
    vxs = readVXSfromLoc(test_1d[i])
    stats_1dr.append(statsForVXS(vxs*16, mode=1, plot=0, thre=thre))

for i in range(len(test_2d)):
    vxs = readVXSfromLoc(test_2d[i])
    stats_2dr.append(statsForVXS(vxs*16, mode=1, plot=0, thre=thre))

for i in range(len(test_rand)):
    vxs = readVXSfromLoc(test_rand[i])
    stats_randr.append(statsForVXS(vxs*16, mode=1, plot=0, thre=thre))

# colors=["#7700ee", "#00ff77", "#e77700"]
# t = np.array(stats_randr)
# plt.scatter(t[:,0],t[:,1], color=colors[2], alpha=0.7)
# t = np.array(stats_1dr)
# plt.scatter(t[:,0],t[:,1], color=colors[0], alpha=0.7)
# t = np.array(stats_2dr)
# plt.scatter(t[:,0],t[:,1], color=colors[1], alpha=0.7)
# plt.legend(["Uncategorized", "1D","2D"])
# plt.xlabel("Density(Molecule counts)")
# plt.ylabel("Connections per Junctions")
# plt.xlim([0,70])
# plt.ylim([0,7])
# plt.title("The stats for real data when thre="+str(thre))
len(stats_randr)
#sns.boxplot(np.array(stats_2dr)[:,4])
labels=["Density", "Connects", "Length Variation", "Percentage of Network","Elastic Energy"]
for i in range(5):
    plt.figure(figsize=[10,5])
    df = pd.Series(np.array(stats_1dr)[:,i], name="1D").to_frame().join(pd.Series(np.array(stats_2dr)[:,i], name="2D"))
    df = df.join(pd.Series(np.array(stats_randr)[:,i], name="Rand"))
    df = df.join(pd.Series(np.array(stats)[:,i], name="Sim 2D"))
    df = df.join(pd.Series(np.array(stats_rand2s)[:,i], name="Sim bi-cluster"))
    df = df.join(pd.Series(np.array(stats_rands)[:,i], name="Sim random"))
    #df = df.join(pd.Series(np.array(stats_2dp)[:,i], name="Sim perfect with noise"))

    # df = df.join(pd.Series(np.array(stats_1ds)[:,i], name="Simulated 1D"))
    # df = df.join(pd.Series(np.array(stats_2ds)[:,i], name="Simulated 2D"))
    # df = df.join(pd.Series(np.array(stats_rands)[:,i], name="Simulated Rand"))

    sns.boxplot(data=df, width = 0.2) 
    plt.title(labels[i]+" comparisons in real data")
vxs_2D = sim2D(size=16*64,space=187)

stats_2dp=[]
for i in range(10):
    vxs = jitter(sim2D(size=64*16, space=187), rand=np.random.randint(5))
    #print(statsForVXS(jitter(vxs, method="noise", noise=np.random.randint(5)), mode=1, plot=0, thre=thre))
    stats_2dp.append(statsForVXS(jitter(vxs, method="noise", noise=np.random.randint(30)), mode=1, plot=0, thre=thre))

#color = cmap.prism
all_links = []
knn_1=[]
knn_2=[]
knn_3=[]
for d in range(80,100,10):
    stats=[]
    for i in range(200):
        vxs = simRandD(box=64*16, d=d, n=45+np.random.randint(15))
        #stats.append(statsForVXS(vxs, mode=1, plot=0, thre=thre))
        dist = distance_matrix(vxs, vxs)
        # s_dist = np.sort(dist, axis=1)
        # knn_1.extend(s_dist[:,1])
        # knn_2.extend(s_dist[:,2])
        # knn_3.extend(s_dist[:,3])
        links = generateLinks(vxs, thre=thre, plot=0)
        lengths = [ x for x in (links*dist).flatten() if x!=0 ]
        all_links.extend(lengths)
        #histLinks(vxs, links, bins=range(1,500,10))
    #t = np.array(stats)
    #plt.scatter(t[:,0],t[:,1], alpha=0.7)
# plt.legend(["d="+str(d) for d in range(80,160,10)])
# plt.xlabel("Density(Molecule counts)")
# plt.ylabel("Connections per Junctions")
# plt.xlim([0,70])
# plt.ylim([0,7])
# plt.title("The stats for simulated data when thre="+str(thre))
plt.figure(figsize=[10,10])
plt.hist(knn_1, bins=range(1,300,10),alpha=0.5)
plt.hist(knn_2, bins=range(1,300,10),alpha=0.5)
plt.hist(knn_3, bins=range(1,300,10),alpha=0.5)
plt.figure(figsize=[10,10])
plt.hist(all_links,bins=range(1,500,10))
vxs = simRand3(64*16, 30, 187, 4)


stats_rands=[]
for i in range(200):
    vxs = simRand(size=64*16, n=45+np.random.randint(15))
    stats_rands.append(statsForVXS(vxs, mode=1, plot=0, thre=thre))
stats_rand2s=[]
for i in range(200):
    vxs = simRand2(size=64*16, n=45+np.random.randint(15))
    stats_rand2s.append(statsForVXS(vxs, mode=1, plot=0, thre=thre))
vxs_1D = sim1D()
stats_1D = statsForVXS(vxs_1D, plot=0, thre=[140,220])
vxs_2D = sim2D(size=16*64,space=93)
stats_2D = statsForVXS(vxs_2D, plot=0, thre=[140,220])
stats_R=[]
for i in range(100):
    vxs_R = simRand(size=16*64,space=187)
    stats_R.append(statsForVXS(vxs_R, plot=0, thre=[140,220]))

stats_R = np.array(stats_R).T
thre=[140,220]
vxs_2D = sim2D(size=16*64, space=93)
stats_loss=[[] for i in range(10)]
for i in range(10):
    for j in range(100):
        t_vxs = jitter(jitter(vxs_2D), method="loss",loss=i*5)
        stats_loss[i].append(statsForVXS(t_vxs, plot=0, thre=thre))
labels = ["Coefficient of Variation of Voronoi", "Average #Edges of Voronoi","Average #Connects per node", "#Tris", "Deviation of 1st Nearest Neighbour"]
for i in range(5):
    plt.figure()
    sns.boxplot(np.array(stats_loss)[:,:,i].T)
    plt.title(labels[i]+" with different loss")
thre=[140,220]
vxs_2D = sim2D(size=16*64, space=93)
stats_noise=[[] for i in range(10)]
for i in range(10):
    for j in range(100):
        t_vxs = jitter(jitter(vxs_2D), method="noise",noise=1+i*5)
        stats_noise[i].append(statsForVXS(t_vxs, plot=0, thre=thre))

labels = ["Coefficient of Variation of Voronoi", "Average #Edges of Voronoi","Average #Connects per node", "#Tris", "Deviation of 1st Nearest Neighbour"]
for i in range(5):
    plt.figure()
    sns.boxplot(np.array(stats_noise)[:,:,i].T)
    plt.title(labels[i]+" with different noise level")
#vxs = vxs*16
links = generateLinks(vxs, thre=[140/16,220/16], plot=0)
connects = Heatmaps(vxs, links=links, method="connects")
tris = Heatmaps(vxs,links=links, method="tris")
v_c = Heatmaps(vxs,method="voronoi_vc")
v_e = Heatmaps(vxs, method="voronoi_edges")
#rk = Heatmaps(vxs,method="ripley", d=[140])
nn_dv = Heatmaps(vxs, method="nn_dv")
density = Heatmaps(vxs)
nn_dv = Heatmaps(vxs, method="nn_dv")
[dmin, dmax] = [int(np.min(density)), int(np.max(density))]
stats_R=[[] for x in range(dmax)]
for counts in range(dmin, dmax):
    for t in range(100):
        vxs_R = np.random.rand(counts,2)*64*16
        stats_R[counts].append(statsForVXS(vxs_R, plot=0, thre=[140,220]))
countss = [x for x in range(dmin,dmax)]
np_R = np.array(stats_R[dmin:dmax+1])
plt.figure(figsize=[10,10])
plt.plot(density.flatten(), connects.flatten(),".")
_=plt.plot(countss,np_R[:,:,2],'o',markerfacecolor="grey", alpha=0.5,markeredgecolor="None")
plt.title("Average Links per Junction vs. Density (Real Data vs. Random Data)")
plt.figure(figsize=[10,10])
plt.plot(density.flatten(), tris.flatten(),".")
_=plt.plot(countss,np_R[:,:,3],'o',markerfacecolor="grey", alpha=0.5, markeredgecolor="None")
plt.title("Number of Linked Triangles vs. Density (Real Data vs. Random Data)")

plt.figure(figsize=[10,10])
plt.plot(density.flatten(), v_c.flatten(),".")
_=plt.plot(countss,np_R[:,:,0],'o',markerfacecolor="grey",alpha=0.5, markeredgecolor="None")
plt.title("Coefficient of Variation for Voronoi vs. Density (Real Data vs. Random Data)")

plt.figure(figsize=[10,10])
plt.plot(density.flatten(), v_e.flatten(),".")
_=plt.plot(countss,np_R[:,:,1],'o',markerfacecolor="grey",alpha=0.5, markeredgecolor="None")
plt.title("Average #Edges for Voronoi vs. Density (Real Data vs. Random Data)")


plt.figure(figsize=[10,10])
plt.plot(density.flatten(), nn_dv.flatten()*256,".")
_=plt.plot(countss,np_R[:,:,4],'o',markerfacecolor="grey",alpha=0.5, markeredgecolor="None")
plt.title("Average Deviation for nearest neighbour vs. Density (Real Data vs. Random Data)")

df = pd.Series(connects.flatten(), name="real").to_frame().join(pd.Series(np_R[:,:,2].flatten(), name="random"))
ax = sns.boxplot(df)
labels=["Voronoi_cv", "#Edges per Voronoi", "#Links per Node", "#Triangle"]
plt.title(labels[2])

plt.figure()
df = pd.Series(tris.flatten(), name="real").to_frame().join(pd.Series(np_R[:,:,3].flatten(), name="random"))
ax = sns.boxplot(df)
labels=["Voronoi_cv", "#Edges per Voronoi", "#Links per Node", "#Triangle"]
plt.title(labels[3])

plt.figure()
df = pd.Series(v_c.flatten(), name="real").to_frame().join(pd.Series(np_R[:,:,0].flatten(), name="random"))
ax = sns.boxplot(df)
labels=["Voronoi_cv", "#Edges per Voronoi", "#Links per Node", "#Triangle"]
plt.title(labels[0])

plt.figure()
df = pd.Series(v_e.flatten(), name="real").to_frame().join(pd.Series(np_R[:,:,1].flatten(), name="random"))
ax = sns.boxplot(df)
labels=["Voronoi_cv", "#Edges per Voronoi", "#Links per Node", "#Triangle"]
plt.title(labels[1])

plt.figure()
df = pd.Series(16*16*nn_dv.flatten(), name="real").to_frame().join(pd.Series(np_R[:,:,4].flatten(), name="random"))
ax = sns.boxplot(df)
labels=["Voronoi_cv", "#Edges per Voronoi", "#Links per Node", "#Triangle","Deviation of First Nearest Neighbour"]
plt.title(labels[4])
plt.boxplot(nn_dv.flatten())
plt.figure(figsize=[10,5])
#ax = sns.boxplot(stats_R.T)
plt.plot(stats_2D[0:4], 'o', c='r',)
labels=["Voronoi_cv", "#Edges per Voronoi", "#Links per Node", "#Triangle"]
ax.set_xticklabels(labels=labels)
vor = Voronoi(vxs)
regions = [vor.vertices[region] for region in vor.regions if -1 not in region and len(region) > 0]
areas = [polygon_area(region) for region in regions]

In [ ]:
### Archived for Temp Report
test1=rendered[650:1500,1000:1850,:]
plt.figure(figsize=[10,10])
plt.imshow(test1)
plt.xticks([], [])
plt.yticks([],[])
plt.savefig("../result/A_c.png",bbox_inches='tight',pad_inches=0.0)

test1=test[650:1500,1000:1850]
plt.figure(figsize=[10,10])
plt.imshow(test1)
plt.xticks([], [])
plt.yticks([],[])
plt.savefig("../result/A.png",bbox_inches='tight',pad_inches=0.0)

prefix = "../data/rendered/soma/1226_5"
img = prefix+".png"
loc_file = prefix+".txt"
regions=[650,1500,1000,1850]
rendered = cv2.imread(img)
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[650:1500,1000:1850]
space=[5,8]
c1,o1,d1 =FFTfilter(test1, space, kernel_size=64, method="bin", threshold=0, sparse=100000, steps=16)
vxs = lmax_loc(None,None,test1,adapt=55, local=7)[1]
links=generateLinks(vxs, thre=15)
xyrange=[regions[3]-regions[2],regions[1]-regions[0]]
connects=Heatmaps(vxs, grid_size=64, steps=16, links=links, method="connects", xyrange=xyrange)
tris = Heatmaps(vxs,links=links, method="tris", xyrange=xyrange)
density=Heatmaps(vxs, xyrange=xyrange)
def plotReps(rep_regions, color="black"):
    cnt=0
    #colors=["#8854d0","#3867d6","#0fb9b1"] 
    #colors=["#9F0D7F","#EA1179","#F79BD3"] #pink purple
    #colors=["#27ae60","#2980b9","#c0392b"] #rgb
    colors=["#2ecc71","#3498db","#e74c3c"] # brighter rgb
    for x, y in rep_regions:
        plt.text(y-100, x+60, str(cnt+1),zorder=4, color=colors[cnt],weight='bold',fontsize=30,path_effects=[pe.withStroke(linewidth=4, foreground="black")])
        plt.gca().add_patch(Rectangle((y-32,x-32),64,64,linewidth=3,edgecolor=colors[cnt],facecolor='none'))
        cnt+=1

a = sumMat(test1, c1, method=method)
b = sumMat(test1.T, tris, method=method)
rep_regions=[np.unravel_index(np.argmax(a, axis=None), a.shape), np.unravel_index(np.argmax(b.T, axis=None), b.T.shape),(320,450)]#,(400,280)]
rep_regions=[(600,150),rep_regions[1],rep_regions[2]]
a = a/np.max(a)
b=b/np.max(b)
for cmap in ["gist_ncar_r"]:
    plt.figure()
    #plt.imshow(test1)
    sns.heatmap(a, vmin=0.5, vmax=np.max(a), cmap=cmap, cbar_kws = dict(use_gridspec=False,location="top",shrink=0.6,pad=-0.04))
    plotReps(rep_regions)
    plt.xticks([], [])
    plt.yticks([],[])
    plt.gca().set_aspect('equal', adjustable='datalim')
    plt.savefig("../result/1d_"+cmap+"2.png",bbox_inches='tight',pad_inches=0.0)

    plt.figure()
    #plt.imshow(test1)
    sns.heatmap(b.T, vmin=0.5, vmax=np.max(b), cmap=cmap,cbar=False)
    plotReps(rep_regions)
    plt.xticks([], [])
    plt.yticks([],[])
    plt.gca().set_aspect('equal', adjustable='datalim')
    plt.savefig("../result/tris_"+cmap+"2.png",bbox_inches='tight',pad_inches=0.0)
# test1=rendered[650:1500,1000:1850,:]
# plt.figure(figsize=[10,10])
# plt.imshow(test1)
# cnt=0
# for x, y in rep_regions:
#      cnt+=1
#      plt.text(y-24, x+20, str(cnt))
#      plt.gca().add_patch(Rectangle((y-32,x-32),64,64,linewidth=5,edgecolor='yellow',facecolor='None'))
# plt.xticks([], [])
# plt.yticks([],[])
# plt.text
# plt.savefig("../result/A_cb.png",bbox_inches='tight',pad_inches=0.0)
colors=["#2ecc71","#3498db","#e74c3c"] 
test1=test[650:1500,1000:1850]
plt.figure(figsize=[10,10])
plt.imshow(test1)
cnt=0
#colors=["#8854d0","#3867d6","#0fb9b1"] 
for x, y in rep_regions:
     plt.text(y-60, x+60, str(cnt+1),zorder=4, color=colors[cnt],weight='bold',fontsize=50,path_effects=[pe.withStroke(linewidth=10, foreground="black")])
     plt.gca().add_patch(Rectangle((y-32,x-32),64,64,linewidth=5,edgecolor=colors[cnt],facecolor='None'))
     cnt+=1

plt.xticks([], [])
plt.yticks([],[])
plt.savefig("../result/Ab.png",bbox_inches='tight',pad_inches=0.0)
a = sumMat(test1, c1, method=method)
b = sumMat(test1.T, tris, method=method)
b=b/np.max(b)-0.3
b[b<0]=0
a=a/np.max(a)-0.4
a[a<0]=0

bb=b.T-a
ab=a-b.T
b[bb.T<0]=0
a[ab<0]=0
def scale(x):
    #return (255*(x/np.max(x))).astype(np.uint8)
    return 255*(x>0.2).astype(np.uint8)
plt.imshow(test1)
plt.imshow(cv2.merge([c.astype(np.uint8),scale(b.T),scale(a)]),alpha=0.3)
plt.imshow(test1)
plt.imshow(cv2.merge([c.astype(np.uint8),scale(bb),scale(ab)]),alpha=0.8)
def plotTwo(a,b, color1,color2):
    color1=hex_to_rgb(color1)
    color2=hex_to_rgb(color2)
    ac=cv2.merge([a*color1[0],a*color1[1],a*color1[2]])
    bc=cv2.merge([b*color2[0],b*color2[1],b*color2[2]])
    ab = ac+bc
    return (ab/np.max(ab)*255).astype(np.uint8)
plt.imshow(test1)
plt.imshow(plotTwo(scale(a),scale(b.T),"#8B008B","#51A5BA"),alpha=0.3)
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16)/255 for i in range(0, lv, lv // 3))
hex_to_rgb("#8B008B")
# a = sumMat(test1, c1, method=method)
# b = sumMat(test1.T, tris, method=method)
# d=255*(a>np.max(a)/7*4).astype(np.uint8)
# e=255*(b.T>np.max(b)/9*4).astype(np.uint8)

import matplotlib.patheffects as pe
# f= e-d
# f[f<0]=0
# for img in [a,b.T,d, e]:
#     plt.figure()
#     plt.imshow(img, vmin=np.max(img)/2, vmax=np.max(img))
#     print(type(img[0,0]))

plt.figure(figsize=[10,10])
c=np.ones(a.shape)
color1="#f368e0"
color2="#54a0ff"

mask = plotTwo(d,f,color1,color2)
w,h = test1.shape
for i in range(w):
     for j in range(h):
          if test1[i,j]>50:
               mask[i,j,:]=[255,255,255]#[test1[i,j],test1[i,j],test1[i,j]]
plt.imshow(mask)#,alpha=0.3)


cnt=0
color="red"
#colors=["#5f27cd","#eb2f06","#fff200"] 
#colors=["#C65BCF","#874CCC","#10439F"] # purple
#colors=["#9F0D7F","#EA1179","#F79BD3"] #pink
for x, y in rep_regions:
     #plt.text(y-60, x+60, str(cnt+1),zorder=4, color="white",fontsize=20,path_effects=[pe.withStroke(linewidth=4, foreground=colors[cnt])])
    
     plt.text(y-60, x+60, str(cnt+1),zorder=4, color=colors[cnt],weight='bold',fontsize=50,path_effects=[pe.withStroke(linewidth=10, foreground="black")])
     plt.gca().add_patch(Rectangle((y-32,x-32),64,64,linewidth=5,edgecolor=colors[cnt],facecolor='None'))
     cnt+=1

plt.xticks([], [])
plt.yticks([],[])

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

legend_elements = [Patch(facecolor=color1, edgecolor=color1,
                         label="1D"), #periodic pattern"),
                    Patch(facecolor=color2, edgecolor=color2,
                         label="2D")] #lattice pattern")     ]
                         

plt.gca().legend(handles=legend_elements, loc='upper right', labelcolor='w',prop={'size':30,'weight':'bold'})
plt.savefig("../result/Result2.png",bbox_inches='tight',pad_inches=0.0)

mask = plotTwo(d,f,color1,color2)
mask.shape
def mergeClosest(vxs, thre=3):
    dist = distance_matrix(vxs,vxs)
    n_vx = len(vxs)
    for i in range(n_vx):
        for j in range(i+1,n_vx):
            if dist[i,j] < thre:
                vxs=vxs[:i]
    return vxs
for x, y in [rep_regions[1]]:
    #rimg = test1[x-32:x+32,y-32:y+32]
    #vxs2 = lmax_loc(loc_file, [1000+y-32,1000+y+32,650+x-32,650+x+32], rimg, adapt=33, plot=0, local=5, verbose=1, sm=3)[1]
    vxs2 = lmax_loc(None, None, rimg, adapt=33, plot=0, local=5, verbose=1, sm=3)[1]
    links = generateLinks(vxs2, thre=15, plot=0)
    plt.figure()
    visualize(vxs2,links,color="#1791FF",linewidth=6)
    plt.imshow(rimg)
    plt.xticks([], [])
    plt.yticks([],[])
    plt.savefig("../result/sub"+str(i)+".png",bbox_inches='tight',pad_inches=0.3,facecolor=colors[1],edgecolor=colors[1])
 
plt.figure(figsize=[10,10])
c=np.ones(a.shape)
plt.imshow(test1)
plt.imshow(cv2.merge([c.astype(np.uint8),d,f]),alpha=0.3)

rep_regions=[np.unravel_index(np.argmax(a, axis=None), a.shape), np.unravel_index(np.argmax(b.T, axis=None), b.T.shape),(320,450)]#,(400,280)]
for x, y in rep_regions:
     plt.gca().add_patch(Rectangle((y-32,x-32),64,64,linewidth=1,edgecolor='r',facecolor='none'))

plt.figure()
for x, y in rep_regions:
    plt.figure()
    plt.imshow(test1[x-32:x+32,y-32:y+32])
i=0
for x, y in rep_regions:
    plt.figure()
    tmp = test1[x-32:x+32,y-32:y+32]
    tmp[tmp>150]=150
    plt.imshow(tmp)
    plt.xticks([], [])
    plt.yticks([],[])
    plt.savefig("../result/sub"+str(i)+".png",bbox_inches='tight',pad_inches=0.3,facecolor=colors[i],edgecolor=colors[i])
    i+=1
plt.figure(figsize=[10,10])
plt.imshow(test1)
#plt.text(30,810,"spectrin",color="white",fontsize=20, fontweight=20)
cnt=0
color="red"
for x, y in rep_regions:
     cnt+=1
     plt.text(y-60, x+60, str(cnt),zorder=4, color=color,fontweight=20,fontsize=20)
     plt.gca().add_patch(Rectangle((y-32,x-32),64,64,linewidth=5,edgecolor=color,facecolor='none'))
plt.xticks([], [])
plt.yticks([],[])
plt.savefig("../result/Ab.png",bbox_inches='tight',pad_inches=0.0)